## Notebook for plotting data atop the catchment area

In [1]:
# Initialisation cell

# For local notebook
# %load_ext autoreload
# %autoreload 2
# %config IPCompleter.greedy=True
# import os
# import sys
# sys.path.append("../") # go to parent dir
# from src.constants import DATA_PATH, PROJECT_PATH

# For Google Colab
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Install if necessary
%pip install geopandas folium seaborn

     |████████████████████████████████| 972kB 6.9MB/s 
     |████████████████████████████████| 14.8MB 332kB/s 
     |████████████████████████████████| 6.5MB 19.3MB/s 


In [3]:
# Import packages
# Data handling
import numpy as np
import pandas as pd
import geopandas as gpd

# Plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns
import folium
import folium.plugins
from IPython.display import IFrame

In [5]:
# Local
# asset_file = os.path.join(DATA_PATH, "Shipston NFM 15.10.xlsx")
# location_file = os.path.join(DATA_PATH, "Gauge Locations.xlsx")

# Google Colab
asset_file = "drive/Shared drives/Shipston-Flood-Risk-Assessment/spatial_data/Shipston NFM 15.10.xlsx"
location_file = "drive/Shared drives/Shipston-Flood-Risk-Assessment/spatial_data/Gauge Locations.xlsx"

# Import data
nfm_assets = pd.read_excel(asset_file, header=1)
gaugeLocations = pd.read_excel(location_file)

# Rename longitude and latitude coords
nfm_assets = nfm_assets.rename(columns={"x": "longitude", "y": "latitude"})

# Get the log10 price (due to varying price orders of magnitude)
nfm_assets["log10_price"] = np.log10(nfm_assets["Installed Cost (£)"] + 0.01)

# Finding all asset types
asset_types = nfm_assets["Asset Type"].unique()
gauge_types = gaugeLocations["Gauge type"].unique()

In [7]:
# Read in catchment data
# boundaryPath = os.path.join(DATA_PATH, "shapefiles/54106.shp")
boundaryPath = "drive/Shared drives/Shipston-Flood-Risk-Assessment/spatial_data/54106.shp"
boundaryShape = gpd.read_file(boundaryPath)
boundaryJson = boundaryShape.to_crs(epsg=4326).to_json()
style_function = lambda x: {'color': '#ff0000','fill': False}
boundaryPlot = folium.features.GeoJson(boundaryJson, style_function)

# Read in Watercourse data
# watercoursePath = os.path.join(DATA_PATH, "shapefiles/WatercourseLink.shp")
# watercoursePath = "drive/My Drive/EA_hackathon/data/WatercourseLink.shp"
# watercourseShape = gpd.read_file(watercoursePath)
# watercourseJson = watercourseShape.to_crs(epsg=4326).to_json()
# watercoursePlot = folium.features.GeoJson(watercourseJson)

# Topography data
# topoPath = "drive/My Drive/EA_hackathon/data/srtm41.kmz"
# topoGrid = gpd.read_file(topoPath)

In [17]:
# Plot mapped data
shipston_map = folium.Map(location=[52.05, -1.670],
    #tiles='Stamen Terrain',
    tiles="https://api.mapbox.com/styles/v1/croydonbrixton/ckg8jcp6c08vb19o089gcrp0b/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiY3JveWRvbmJyaXh0b24iLCJhIjoiY2tmYTJzZDJiMHJ4YjJ5cXNzMjQzYXd5MSJ9.zUToqkuiUUEdWavpHmXpzQ",
    zoom_start=12.3,
    attr="Mapbox Control Room"
)

asset_color = {}
for asset_type, color in zip(asset_types, sns.color_palette().as_hex()):
    asset_color[asset_type] = color

for i, asset in nfm_assets.iterrows():
  folium.features.RegularPolygonMarker(
      location = (asset.latitude, asset.longitude),
      popup = f"{asset['Asset Type']} {i}: {asset['Storage Created (m3)']} m3",
      number_of_sides = 3,
      radius = 6,
      weight = 1,
      fill_opacity = 0.5,
      rotation = 30, 
      color=asset_color[asset["Asset Type"]],
      fill_color=asset_color[asset["Asset Type"]]).add_to(shipston_map)

gauge_color = {}
for gauge_type, color in zip(gauge_types, sns.color_palette().as_hex()):
    gauge_color[gauge_type] = color

for j, location in gaugeLocations.iterrows():
  folium.features.RegularPolygonMarker(
      location = (location.Latitude,location.Longitude),
      popup = f"{location['Gauge name']}, data: {location['Timeframe']}",
      number_of_sides = 3,
      radius = 12,
      weight = 1,
      fill_opacity = 0.5,
      rotation = 90, 
      color=gauge_color[location["Gauge type"]],
      fill_color=gauge_color[location["Gauge type"]]).add_to(shipston_map)

legend_html = """
<div style="position:fixed;
     top: 40px; 
     left: 50px; 
     width: 300px; 
     height: 230px; 
     border:2px solid grey; 
     z-index: 9999;
     font-size:16px;">
     &nbsp;<b>NFM Assets:</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:blue"></i>&nbsp;<b>Leaky Barriers</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:orange"></i>&nbsp;<b>Offline Storage Areas</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:green"></i>&nbsp;<b>Run-off Pathway Management</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:purple"></i>&nbsp;<b>Wetland Restoration</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:red"></i>&nbsp;<b>Woodland Creation</b><br>
     &nbsp<b>Measurement Stations:</b><br>
     &nbsp;<i class="fa fa-diamond fa-1x" style="color:blue"></i>&nbsp;<b>River flow</b><br>
     &nbsp;<i class="fa fa-diamond fa-1x" style="color:orange"></i>&nbsp;<b>Precipitation - recording</b><br>
     &nbsp;<i class="fa fa-diamond fa-1x" style="color:green"></i>&nbsp;<b>Precipitation - manual</b><br>
</div>"""
shipston_map.get_root().html.add_child(folium.Element(legend_html))
shipston_map.add_child(boundaryPlot)
#shipston_map.add_child(watercoursePlot)
# Display Map
display(shipston_map)